# Sections

This section reviews more advanced functionality for creating and manipulating `pystrat.Section` objects and plotting stratigraphic sections.

The example data utilized here comes from the references listed at the end of the guide.

## Units

In [ ]:
# read in your lithostratigraphy data
litho_df = pd.read_csv('example-data/lithostratigraphy.csv')

# create a pystrat Section from the lithostratigraphy data
section_01 = pystrat.Section(litho_df['THICKNESS'],           # unit thicknesses
                             litho_df['FACIES'],              # unit facies
                             units=litho_df[['GROUP', 'FORMATION']].values) # group and formation names

## Advanced Styling

### Annotations

### Style Compatibility

## Stratigraphic Data

## Helper Functions

The `pystrat.Section` class offers various helpful functions relevant for stratigraphic sections.

## References

> Swanson-Hysell, N.L., Maloof, A.C., Condon, D.J., Jenkin, G.R., Alene, M., Tremblay, M.M., Tesema, T., Rooney, A.D., and Haileab, B., 2015, Stratigraphy and geochronology of the Tambien Group, Ethiopia: Evidence for globally synchronous carbon isotope change in the Neoproterozoic: Geology, v. 43, p. 323-326, https://doi.org/10.1130/G36347.1.

> MacLennan, S.A., Park, Y., Swanson-Hysell, N.L., Maloof, A.C., Schoene, B., Gebreslassie, M., Antilla, E., Tesema, T., Alene, M., and Haileab, B., 2018, The arc of the Snowball: U-Pb dates constrain the Islay anomaly and the initiation of the Sturtian glaciation: Geology, v. 46, p. 539-542, https://doi.org/10.1130/G40171.1.

>Park, Y., Swanson-Hysell, N.L., MacLennan, S.A., Maloof, A.C., Gebreslassie, M., Tremblay, M.M., Schoene, B., Alene, M., Anttila, E.S.C., Tesema, T., Condon, D.J., Haileab, B., 2020, The lead-up to the Sturtian Snowball Earth: Neoproterozoic chemostratigraphy time-calibrated by the Tambien Group of Ethiopia: GSA Bulletin, vol. 132, pp. 1119–1149, https://doi.org/10.1130/B35178.1